In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
from scipy.special import roots_legendre
from scipy.sparse.linalg import eigs

In [ ]:
# Constants
Np = 100
Nphi = 40
pi = np.pi
ONE = 1.0
ZERO = 0.0
Tel_deuteron = 1.0e-10
mass = 1.0
m = 0  # Orbital quantum number (for angular momentum)
n = 0  # Principal quantum number, must be >= msa
E_exact = -ONE / (n + 0.5)**2  # Exact energy for comparison
BE2_1 = E_exact * 0.95  # Slightly lower bound for energy estimate
BE2_2 = E_exact * 1.05  # Slightly upper bound for energy estimate

In [ ]:
def hyperbolic(Np):
    """
    Generate momentum points and weights using a hyperbolic distribution.
    """
    X, dX = roots_legendre(Np)
    P = (1.0 + X) / (1.0 - X)
    dP = 2.0 / (1.0 - X)**2 * dX
    return P, dP

def linear(Nphi, a, b):
    """
    Generate linearly spaced angular points and weights.
    """
    X, dX = roots_legendre(Nphi)
    x = (b - a) / 2 * X + (b + a) / 2
    dx = (b - a) / 2 * dX
    return x, dx

In [ ]:
def compute_V_2B(Np, Nphi, m, p, xp, phi, dphi):
    """
    Compute the potential matrix V_2B and integrate over the angular coordinate phi.
    """
    V_2B = np.zeros((Np, Np, Nphi))
    for ip in range(Np):
        for ipp in range(Np):
            for ixp in range(Nphi):
                q = np.sqrt(p[ip]**2 + p[ipp]**2 - 2 * p[ip] * p[ipp] * xp[ixp])
                V_2B[ip, ipp, ixp] = -1.0 / (q * pi)

    V_2B_2d = np.zeros((Np, Np))
    cos_terms = np.cos(m * phi)
    for ipp in range(Np):
        for ip in range(Np):
            V_2B_2d[ip, ipp] = np.sum(dphi * V_2B[ip, ipp, :] * cos_terms)

    return V_2B_2d

In [ ]:
def eigenvalue(Mmatrix):
    """
    This function calculates a few eigenvalues and eigenvectors of the matrix Mmatrix.

    Args:
        Mmatrix (numpy.ndarray): A square matrix from which to find eigenvalues and eigenvectors.

    Returns:
        WF (numpy.ndarray): The eigenvectors corresponding to the largest eigenvalue.
        Lambda (float): The largest eigenvalue.
    """
    # Determine the size of the matrix
    dim_Mmatrix = Mmatrix.shape[0]

    # Specify the number of eigenvalues and eigenvectors to find
    # NEV: number of eigenvalues,
    # NCV: number of Arnoldi vectors, must be greater than NEV
    NEV = 10
    NCV = 2 * NEV # A larger NCV can accelerate convergence
    # Use 'LM' (Largest Magnitude) or 'LR' (Largest Real part) as needed
    WHICH = 'LR'
    # Find NEV eigenvalues with the largest magnitude
    eigenvalues, eigenvectors = eigs(Mmatrix, k=NEV, which=WHICH, ncv=NCV)
    # Target eigenvalue near 1
    target_eigenvalue = 1.0
    idx_min = np.argmin(np.abs(eigenvalues - target_eigenvalue))
    Lambda = eigenvalues[idx_min]
    WF = np.abs(eigenvectors[:, idx_min])

    return WF, Lambda

In [ ]:
def LS_2D(Np, pp, dp, V_2B):
    """
    Solve the Lippmann-Schwinger equation in 2D for the given potential matrix.

    Args:
        Np (int): Number of momentum grid points.
        pp (np.ndarray): Array of momentum values, shape (Np,).
        dp (np.ndarray): Differential weights associated with pp, shape (Np,).
        V_2B (np.ndarray): Non-relativistic potential matrix, shape (Np, Np).

    Returns:
        float: The energy eigenvalue closest to the expected bound state energy.
        np.ndarray: Normalized eigenvector corresponding to the computed eigenvalue.
    """
    Ne = 20
    BE_ary = np.zeros(Ne)
    Lam_ary = np.zeros(Ne)
    BE_ary[0] = BE2_1
    BE_ary[1] = BE2_2

    for ie in range(Ne):
        if ie > 1:
            BE_ary[ie] = BE_ary[ie-1] + (1 - Lam_ary[ie-1]) / (Lam_ary[ie-2] - Lam_ary[ie-1]) * (BE_ary[ie-2] - BE_ary[ie-1])
        BE2 = BE_ary[ie]
        Mmatrix = np.zeros((Np, Np))
        for ip in range(Np):
            for ipp in range(Np):
                denom = (BE2 - pp[ip]**2 / mass)
                if denom != 0:
                    Mmatrix[ip, ipp] = dp[ipp] * pp[ipp] * V_2B[ip, ipp] / denom

        WF, Lambda = eigenvalue(Mmatrix)
        WF /= np.sqrt(np.sum(WF**2 * dp * pp))

        Lam_ary[ie] = Lambda
        print(f"Iteration {ie} | BE2 = {BE2}, Lambda = {Lambda}")

        if np.abs(Lambda - 1.0) < Tel_deuteron:
            break

    return BE2, WF

In [ ]:
def main():
    """
    Main function to orchestrate the computation and visualization.
    """
    p, dp = hyperbolic(Np)
    phi, dphi = linear(Nphi, 0, 2.0 * pi)
    xp = np.cos(phi)
    V_2B_2d = compute_V_2B(Np, Nphi, m, p, xp, phi, dphi)
    BE2, WF = LS_2D(Np, p, dp, V_2B_2d)
    rel_error = np.abs((BE2 - E_exact) / E_exact) * 100

    print(f'Final BE2: {BE2}')
    print(f'Exact BE2: {E_exact}')
    print(f'Rel % Difference: {rel_error}')

    plt.figure(figsize=(10, 6))
    plt.loglog(p, WF, 'bo-', label='Wave Function')
    plt.xlabel('Relative Momentum p (log scale)')
    plt.ylabel('Wave Function (log scale)')
    plt.title('Log-Log Plot of Wave Function vs. Relative Momentum')
    # plt.grid(True, which="both", ls="--")
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
from scipy.special import roots_legendre
from scipy.sparse.linalg import eigs

In [ ]:
# Constants
Np = 200
Nphi = 40
pi = np.pi
ONE = 1.0
ZERO = 0.0
Tel_deuteron = 1.0e-10
mass = 1.0
m = 0  # Orbital quantum number (for angular momentum)

In [ ]:
def hyperbolic(Np):
    """
    Generate hyperbolic distribution points and their weights using Legendre roots.
    Returns:
        (P, dP): Tuple of numpy arrays of shape (Np,)
                 P is the transformed momentum points
                 dP is the corresponding weights
    """
    X, dX = roots_legendre(Np)
    P = (1.0 + X) / (1.0 - X)
    dP = 2.0 / (1.0 - X)**2 * dX
    return P, dP

def linear(Nphi, a, b):
    """
    Generate linear distribution points and their weights for variables over an interval [a, b].
    Args:
        Nphi (int): Number of points in phi.
        a (float): Start of the interval.
        b (float): End of the interval.

    Returns:
        (x, dx): Tuple of numpy arrays of shape (Nphi,)
                 x is the linearly mapped points from [-1, 1] to [a, b]
                 dx is the corresponding weights
    """
    X, dX = roots_legendre(Nphi)
    x = (b - a) / 2 * X + (b + a) / 2
    dx = (b - a) / 2 * dX
    return x, dx


In [ ]:
def eigenvalue(Mmatrix):
    """
    This function calculates a few eigenvalues and eigenvectors of the matrix Mmatrix.

    Args:
        Mmatrix (numpy.ndarray): A square matrix from which to find eigenvalues and eigenvectors.

    Returns:
        WF (numpy.ndarray): The eigenvectors corresponding to the largest eigenvalue.
        Lambda (float): The largest eigenvalue.
    """
    # Determine the size of the matrix
    dim_Mmatrix = Mmatrix.shape[0]

    # Specify the number of eigenvalues and eigenvectors to find
    # NEV: number of eigenvalues,
    # NCV: number of Arnoldi vectors, must be greater than NEV
    NEV = 10
    NCV = 2 * NEV # A larger NCV can accelerate convergence
    # Use 'LM' (Largest Magnitude) or 'LR' (Largest Real part) as needed
    WHICH = 'LR'
    # Find NEV eigenvalues with the largest magnitude
    eigenvalues, eigenvectors = eigs(Mmatrix, k=NEV, which=WHICH, ncv=NCV)
    # Target eigenvalue near 1
    target_eigenvalue = 1.0
    idx_min = np.argmin(np.abs(eigenvalues - target_eigenvalue))
    Lambda = eigenvalues[idx_min]
    WF = np.abs(eigenvectors[:, idx_min])

    return WF, Lambda

In [ ]:
def compute_V_2B(Np, Nphi, m, p, xp, phi, dphi):
    """
    Computes the 3D array V_2B (for potential calculations) and integrates it
    over phi to form a 2D array V_2B_2d.

    Args:
        Np (int): Number of p points
        Nphi (int): Number of phi points
        m: state quantum number
        p (np.ndarray): 1D array of momentum values of length Np
        xp (np.ndarray): 1D array of cos(phi) values of length Nphi
        dphi (np.ndarray): 1D array of differential phi weights of length Nphi

    Returns:
        V_2B_2d (np.ndarray): 2D array of shape (Np, Np), the integrated potential over phi
    """
    # Initialize the V_2B 3D array
    V_2B = np.zeros((Np, Np, Nphi))

    # Populate the V_2B array
    for ip in range(Np):
        for ipp in range(Np):
            for ixp in range(Nphi):
                q = np.sqrt(p[ip]**2 + p[ipp]**2
                            - 2.0 * p[ip] * p[ipp] * xp[ixp])
                # Potential V2B_q = -1 / (q*pi)
                V_2B[ip, ipp, ixp] = -1.0 / (q * pi)

    # Integrate V_2B over phi to produce V_2B_2d
    V_2B_2d = np.zeros((Np, Np))
    cos_terms = np.cos(m * phi)  # This computes the cosine term for all phi values

    for ipp in range(Np):
        for ip in range(Np):
          # phi integration
           V_2B_2d[ip, ipp] = np.sum(dphi * V_2B[ip, ipp, :] * cos_terms)

    return V_2B_2d

In [ ]:
def LS_2D(Np, pp, dp, V_NR, BE2_1, BE2_2):
    """
    Demonstration routine for computing eigenvalues using the 2D LS approach.

    Args:
        Np (int): Number of momentum points
        pp (np.ndarray): Momentum array of shape (Np,)
        dp (np.ndarray): Weights array associated with pp
        V_NR (np.ndarray): 2D potential array V_NR of shape (Np, Np)

    Returns:
        (BE2, WF): Tuple of (float, np.ndarray)
                   BE2 is the last computed energy
                   WF is the corresponding wave function array of shape (Np,)
    """
    Ne = 20
    BE_ary = np.zeros(Ne)
    Lam_ary = np.zeros(Ne)
    BE_ary[0] = BE2_1
    BE_ary[1] = BE2_2

    for ie in range(Ne):
        # Extrapolation-like step if needed:
        if ie > 1:
            BE_ary[ie] = (BE_ary[ie-1]
                          + (1 - Lam_ary[ie-1])/(Lam_ary[ie-2] - Lam_ary[ie-1])
                          * (BE_ary[ie-2] - BE_ary[ie-1]))
        BE2 = BE_ary[ie]

        # Build the matrix
        Mmatrix = np.zeros((Np, Np))
        for ip in range(Np):
            for ipp in range(Np):
                denom = (BE2 - pp[ip]**2 / mass)
                if denom != 0:
                    Mmatrix[ip, ipp] = (dp[ipp] * pp[ipp] * V_NR[ip, ipp] / denom)

        WF, Lambda = eigenvalue(Mmatrix)
        # Normalize WF
        WF /= np.sqrt(np.sum(WF**2 * dp * pp))

        Lam_ary[ie] = np.real(Lambda)
        print(f"Iteration {ie} | BE2 = {BE2}, Lambda = {Lambda}")

        if np.abs(Lambda - 1.0) < Tel_deuteron:
            break

    return BE2, WF


In [ ]:
# -----------------------
# Main Program
# -----------------------
def main():
    p, dp = hyperbolic(Np)
    phi, dphi = linear(Nphi, 0, 2.0 * pi)
    xp = np.cos(phi)
    WF_matrix = []

    # Loop over values of n from m up to 4
    for n in range(m, 5):  # m to 4, inclusive
        E_exact = -ONE / (n + 0.5)**2  # Calculate the exact energy for each n
        BE2_1 = E_exact * 0.95  # Lower bound for energy estimate
        BE2_2 = E_exact * 1.05  # Upper bound for energy estimate

        V_2B_2d = compute_V_2B(Np, Nphi, m, p, xp, phi, dphi)
        BE2, WF = LS_2D(Np, p, dp, V_2B_2d, BE2_1, BE2_2)
        rel_error = np.abs((BE2 - E_exact) / E_exact) * 100
        WF_matrix.append(WF)

        print(f'For n = {n}:')
        print(f'Computed BE2: {BE2}')
        print(f'Exact Energy: {E_exact}')
        print(f'Relative Error: {rel_error}%\n')

        plt.figure(figsize=(10, 6))  # Initialize the plot for all wave functions
        plt.figure(figsize=(10, 6))
        plt.loglog(p, WF, label=f'm={m}, n={n} Wave Function')
        plt.xlabel('Relative Momentum p (log scale)')
        plt.ylabel('Wave Function (log scale)')
        plt.title(f'Wave Function vs. Momentum for n={n}, m={m}')
        # plt.grid(True, which="both", ls="--")
        plt.legend()
        plt.show()

        plt.figure(figsize=(10, 6))
    for i, WF in enumerate(WF_matrix):
        plt.loglog(p, WF, label=f'n={i}')
    plt.xlabel('Relative Momentum p (log scale)')
    plt.ylabel('Wave Function (log scale)')
    plt.title(f'Comparison of Wave Functions for Different n Values and m={m}')
    # plt.grid(True, which="both", ls="--")
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()